<a href="https://colab.research.google.com/github/Annieemmanuel/ML-engineer-take-home-research-project/blob/main/MTailor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install bs4 lxml kaggle torchmetrics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 9.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=d0a454b380e7ccef6fef432be8249dd12c0ea2b8fdbcd0183800aa56358ae229
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [3]:
import os
os.environ['KAGGLE_USERNAME'] = 'bilalyousaf0014'
os.environ['KAGGLE_KEY'] = '11031bc21c5e3ec23585dbe17dc4267d'


In [6]:
!kaggle datasets download -d bilalyousaf0014/ml-engineer-assessment-dataset

 99% 78.0M/78.6M [00:03<00:00, 29.9MB/s]
100% 78.6M/78.6M [00:03<00:00, 22.5MB/s]


In [7]:
! unzip /content/ml-engineer-assessment-dataset.zip

Archive:  /content/ml-engineer-assessment-dataset.zip
  inflating: assessment_dataset/images/00001.jpeg  
  inflating: assessment_dataset/images/00008.jpeg  
  inflating: assessment_dataset/images/00017.jpeg  
  inflating: assessment_dataset/images/00022.jpeg  
  inflating: assessment_dataset/images/00048.jpeg  
  inflating: assessment_dataset/images/00055.jpeg  
  inflating: assessment_dataset/images/001.jpeg  
  inflating: assessment_dataset/images/1.jpeg  
  inflating: assessment_dataset/images/1001524.jpeg  
  inflating: assessment_dataset/images/1005343.jpeg  
  inflating: assessment_dataset/images/1049854.jpeg  
  inflating: assessment_dataset/images/1072860.jpeg  
  inflating: assessment_dataset/images/1120419.jpeg  
  inflating: assessment_dataset/images/1146885.jpeg  
  inflating: assessment_dataset/images/2.jpeg  
  inflating: assessment_dataset/images/3.jpeg  
  inflating: assessment_dataset/images/4.jpeg  
  inflating: assessment_dataset/images/5.jpeg  
  inflating: assessm

In [8]:
import os
import torch
import torch.nn as nn
import numpy as np

from torchvision.models import resnet18, ResNet18_Weights

In [41]:
class Model(nn.Module):

    def __init__(self, num_species=9):
        super(Model, self).__init__()
        pretrained_model = resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(pretrained_model.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.object_fc = nn.Linear(512, 2)
        self.cat_dog_fc = nn.Linear(512, 2)
        self.species_fc = nn.Linear(512, num_species)
        self.bbox_fc = nn.Linear(512, 4)

    def forward(self, x):
        out_backbone = self.backbone(x)
        out_avgpool = self.avgpool(out_backbone)
        out_avgpool = out_avgpool.view(out_avgpool.size(0), -1)
        
        object_output = self.object_fc(out_avgpool)
        cat_dog_output = self.cat_dog_fc(out_avgpool)
        species_output = self.species_fc(out_avgpool)
        bbox_output = self.bbox_fc(out_avgpool)

        return {
            "bbox": bbox_output,
            "object": object_output,
            "cat_or_dog": cat_dog_output,
            "species": species_output
        }


In [10]:
train_list = np.load('/content/assessment_dataset/train_list.npy', allow_pickle=True).tolist()
val_list = np.load('/content/assessment_dataset/val_list.npy', allow_pickle=True).tolist()


In [42]:
from bs4 import BeautifulSoup

def read_xml_file(path):
    with open(path, 'r') as f:
        data = f.read()
    bs_data = BeautifulSoup(data, 'xml')
    return {
        "cat_or_dog": bs_data.find("name").text,
        "xmin": int(bs_data.find("xmin").text),
        "ymin": int(bs_data.find("ymin").text),
        "xmax": int(bs_data.find("xmax").text),
        "ymax": int(bs_data.find("ymax").text),
        "specie": "_".join(path.split(os.sep)[-1].split("_")[:-1])
    }

In [57]:
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, dataset_path, images_list, train=False):
        self.dataset_path = dataset_path
        self.images_list = images_list
        self.train = train

        # Define data augmentation transforms
        if self.train:
            self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(30),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, index):
    # Load image and label
       image_name = self.images_list[index]
       image_path = os.path.join(self.dataset_path, "images", f"{image_name}.jpg")
       label_path = os.path.join(self.dataset_path, "labels", f"{image_name}.xml")

       if not os.path.exists(image_path):
        print(f"Image file does not exist: {image_path}")
        return None

       if not os.path.exists(label_path):
        print(f"Label file does not exist: {label_path}")
        return None

        # Check if image is not None
        

       image = Image.open(image_path)
       label = read_xml_file(label_path)

    # Apply preprocessing and data augmentation
       image = self.transform(image)

       if image is None:
            # Handle the error gracefully by returning an empty dictionary
            return {}
        

       return image, label


    

In [43]:
from PIL import Image

import torchvision.transforms as transforms


class CustomDataset():

  def __init__(self, dataset_path, images_list, train=False):
    self.preprocess = None

    image_folder_path = os.path.join(dataset_path, "images")
    label_folder_path = os.path.join(dataset_path, "labels")

    for path in os.listdir(image_folder_path):
        name = path.split(os.sep)[-1].split(".")[0]
        if name in images_list:
          xml_path = os.path.join(label_folder_path, name+".xml")
          xml_data = read_xml_file(xml_path)

  def __len__(self):
    return None

  def __getitem__(self, index):
    image = self.preprocess(image)
    labels = None
    return image, label

In [58]:
from torch.utils.data import DataLoader

train_list = list(train_list)
training_dataset = CustomDataset("/assessment_dataset", images_list=train_list)
training_loader = DataLoader(training_dataset, batch_size=16, shuffle=True, num_workers=2)


In [59]:
import torchmetrics
import torch.nn.functional as F


def test(model, val_loader):

  def __tl(x):
    return x.tolist()

  def __tn(x):
    return x.detach().cpu().numpy()

  def __tnl(x):
    return (x.detach().cpu().numpy()).tolist()

  def post_process_object(x):
    return torch.where(x > 0.5, 1.0, 0.0).squeeze(1)

  def post_process_cat_or_dog(x):
    return torch.where(x > 0.5, 1.0, 0.0).squeeze(1)

  def post_process_specie(x):
    return torch.argmax(x, dim=1)

  def post_process_xmin(x):
    return x

  def post_process_ymin(x):
    return x

  def post_process_xmax(x):
    return x

  def post_process_ymax(x):
    return x

  def iou(
      out_xmin,
      out_ymin,
      out_xmax,
      out_ymax,
      lab_xmin,
      lab_ymin,
      lab_xmax,
      lab_ymax
    ):
    min_box_x = torch.max(out_xmin, lab_xmin)
    min_box_y = torch.max(out_ymin, lab_ymin)
    max_box_x = torch.min(out_xmax, lab_xmax)
    max_box_y = torch.min(out_ymax, lab_ymax)
    # print(out_xmin, out_ymin, out_xmax, out_ymax)
    area_over = (
        torch.max(torch.tensor([0]), (max_box_x-min_box_x+torch.tensor(1.0)))
        *torch.max(torch.tensor([0]), max_box_y-min_box_y+torch.tensor(1.0))
      )
    area_box_1 = (out_xmax-out_xmin+1)*(out_ymax-out_ymin+1)
    area_box_2 = (lab_xmax-lab_xmin+1)*(lab_ymax-lab_ymin+1)
    iou = torch.abs(area_over / (area_box_1 + area_box_2 - area_over))
    iou_val = torch.mean(iou)
    return iou_val

  metric_object = torchmetrics.F1Score(task="binary")
  metric_cat_or_dog = torchmetrics.F1Score(task="binary")
  metric_specie = torchmetrics.F1Score(task="multiclass", num_classes=9)

  output_list = {
      "object": [],
      "cat_or_dog": [],
      "specie": [],
      "xmin": [],
      "ymin": [],
      "xmax": [],
      "ymax": [],
  }
  labels_list = {
      "object": [],
      "cat_or_dog": [],
      "specie": [],
      "xmin": [],
      "ymin": [],
      "xmax": [],
      "ymax": [],
  }

  for i, data in enumerate(val_loader):
    inputs, labels = data
    if torch.cuda.is_available():
      inputs = inputs.cuda()
      # for key, value in labels.items():
      #     labels[key] = labels[key].cuda()

    # Make predictions for this batch
    outputs = model(inputs)

    is_object = __tnl(labels["have_object"])
    width = __tn(labels["width"])
    height = __tn(labels["height"])
    output_list["object"].extend(__tnl(post_process_object(outputs["object"])))
    labels_list["object"].extend(__tnl(labels["have_object"]))

    if is_object[0] == 1.0:
      output_list["cat_or_dog"].extend(
        __tnl(post_process_cat_or_dog(outputs["cat_or_dog"]))
      )
      labels_list["cat_or_dog"].extend(
        __tnl(labels["cat_or_dog"])
      )
      output_list["specie"].extend(
        __tnl(post_process_specie(outputs["specie"]))
      )
      labels_list["specie"].extend(__tnl(labels["specie"]))
      output_list["xmin"].extend(
        __tl(__tn(post_process_xmin(outputs["bbox"][:, 0]))*width)
      )
      labels_list["xmin"].extend(__tl(__tn(labels["xmin"])*width))
      output_list["ymin"].extend(
          __tl(__tn(post_process_ymin(outputs["bbox"][:, 1]))*height)
      )
      labels_list["ymin"].extend(
          __tl(__tn(labels["ymin"])*height)
      )
      output_list["xmax"].extend(
          __tl(__tn(post_process_xmax(outputs["bbox"][:, 2]))*width)
      )
      labels_list["xmax"].extend(__tl(__tn(labels["xmax"])*width))
      output_list["ymax"].extend(__tl(__tn(post_process_ymax(outputs["bbox"][:, 3]))*height))
      labels_list["ymax"].extend(__tl(__tn(labels["ymax"])*height))
  
  f1_score_object = metric_object(torch.tensor(output_list["object"]), torch.tensor(labels_list["object"]))
  f1_score_cat_or_dog = metric_cat_or_dog(torch.tensor(output_list["cat_or_dog"]), torch.tensor(labels_list["cat_or_dog"]))
  f1_score_specie = metric_specie(torch.tensor(output_list["specie"]), torch.tensor(labels_list["specie"]))
  mean_iou = iou(
      torch.tensor(output_list["xmin"]),
      torch.tensor(output_list["ymin"]),
      torch.tensor(output_list["xmax"]),
      torch.tensor(output_list["ymax"]),
      torch.tensor(labels_list["xmin"]),
      torch.tensor(labels_list["ymin"]),
      torch.tensor(labels_list["xmax"]),
      torch.tensor(labels_list["ymin"]),
  )
  return f1_score_object, f1_score_cat_or_dog, f1_score_specie, mean_iou
################################## HELPER CODE PROVIDED BY HIRING TEAM ##################################

# Initialize Model and Optimizer
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Initialize Loss Functions
have_object_loss = nn.BCEWithLogitsLoss()
specie_loss = nn.CrossEntropyLoss()
cat_or_dog_loss = nn.CrossEntropyLoss()
bbox_loss = nn.MSELoss()
xmin_loss = nn.MSELoss()
ymin_loss = nn.MSELoss()
xmax_loss = nn.MSELoss()
ymax_loss = nn.MSELoss()



def train_one_epoch(epoch_index, tb_writer, training_loader):
    running_loss = 0.
    last_loss = 0.

    # Set model to train mode
    model.train()

    # Loop through the batches
    for i, data in enumerate(training_loader):
        # Clear the gradients
        optimizer.zero_grad()

        # Every data instance is an input + label pair
        inputs, labels= data

        # Move the inputs and labels to the device
        inputs = inputs.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss_have_object = have_object_loss(outputs["object"], labels["have_object"])
        loss_specie = specie_loss(outputs["specie"], labels["specie"])
        loss_cat_or_dog = cat_or_dog_loss(outputs["cat_or_dog"], labels["cat_or_dog"])
        loss_bbox = bbox_loss(outputs["bbox"], labels["bbox"])
        loss_xmin = xmin_loss(outputs["bbox"][:, 0], labels["bbox"][:, 0])
        loss_ymin = ymin_loss(outputs["bbox"][:, 1], labels["bbox"][:, 1])
        loss_xmax = xmax_loss(outputs["bbox"][:, 2], labels["bbox"][:, 2])
        loss_ymax = ymax_loss(outputs["bbox"][:, 3], labels["bbox"][:, 3])

        loss = loss_have_object + loss_specie + loss_cat_or_dog + loss_bbox + loss_xmin + loss_ymin + loss_xmax + loss_ymax

        # Compute gradients and update the parameters
        loss.backward()
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 0:
            last_loss = running_loss / 10 # loss per batch
            running_loss = 0.

    return last_loss
epochs=10

for i in range(epochs):
    # Initialize Loss Functions
    have_object_loss = None
    specie_loss = None
    cat_or_dog_loss = None
    bbox_loss = None # Not necessary you need to apply function to all coordinates together, You can have separete loss functions for all coordinates too
    # Below or Above
    xmin_loss = None
    ymin_loss = None
    xmax_loss = None
    ymax_loss = None


    epoch_loss = train_one_epoch(i, None, training_loader)
    print(f' Epoch {i} Loss : {epoch_loss}')

    torch.save("model.pth", model.state_dict())
    metrics = test(model)
    print(metrics)

Image file does not exist: /assessment_dataset/images/pomeranian_130.jpg
Image file does not exist: /assessment_dataset/images/american_pit_bull_terrier_166.jpg
Image file does not exist: /assessment_dataset/images/n01631663_eft.jpg
Image file does not exist: /assessment_dataset/images/Birman_111.jpg
Image file does not exist: /assessment_dataset/images/pomeranian_188.jpgImage file does not exist: /assessment_dataset/images/chihuahua_143.jpg
Image file does not exist: /assessment_dataset/images/Abyssinian_197.jpg
Image file does not exist: /assessment_dataset/images/Persian_15.jpg

Image file does not exist: /assessment_dataset/images/Persian_181.jpg
Image file does not exist: /assessment_dataset/images/Persian_107.jpg
Image file does not exist: /assessment_dataset/images/Persian_162.jpgImage file does not exist: /assessment_dataset/images/american_pit_bull_terrier_122.jpg


Image file does not exist: /assessment_dataset/images/american_pit_bull_terrier_136.jpgImage file does not exist

TypeError: ignored

In [62]:
import torchmetrics
import torch.nn.functional as F
import torch

def train(model, train_loader, val_loader, optimizer, criterion, num_epochs, save_path):

    best_val_loss = float("inf")
    best_val_acc = 0.0

    for epoch in range(num_epochs):

        # Training phase
        model.train()
        train_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                for key, value in labels.items():
                    labels[key] = labels[key].cuda()

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        for i, data in enumerate(val_loader):
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                for key, value in labels.items():
                    labels[key] = labels[key].cuda()

            # Forward pass
            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

            # Compute validation accuracy
            predicted_labels = torch.argmax(outputs, dim=1)
            val_acc += (predicted_labels == labels["target"]).sum().item()

        val_loss /= len(val_loader)
        val_acc /= len(val_loader.dataset)

        # Check if validation loss or accuracy is the best so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path + "best_loss.pt")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "/content/assessment_dataset/" + "best_acc.pt")

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    print("Finished training.")


In [68]:
import os
import torch
from PIL import Image, ImageDraw
from torchvision.transforms import functional as F

def visualize(model_weights, image_folder_path, output_folder="output"):
    model = Model()
    model.load_state_dict(torch.load(model_weights, map_location=torch.device('cpu')))
    model.eval()

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_details = {}

    for image_name in os.listdir(image_folder_path):
        if image_name.endswith(".jpg") or image_name.endswith(".jpeg"):
            try:
                image = Image.open(os.path.join("/content/assessment_dataset/images", image_name+".jpg"))
            except:
                print(f"Could not open image: {image_name}")
                continue

            image_tensor = F.to_tensor(image)
            image_tensor = F.normalize(image_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            output = model(image_tensor.unsqueeze(0))

            output_class = torch.argmax(output["class"]).item()
            output_score = torch.max(torch.softmax(output["class"], dim=1)).item()

            if output_score < 0.5:
                has_object = False
                cat_or_dog = "NA"
                specie = "NA"
                xmin = ymin = xmax = ymax = 0
            else:
                has_object = True
                cat_or_dog = "cat" if output_class == 0 else "dog"
                specie = "persian" if output["species"][0][0] > output["species"][0][1] else "siamese"
                xmin, ymin, xmax, ymax = [int(coord) for coord in output["bbox"].tolist()[0]]

            image_details[image_name] = {
                "has_object": has_object,
                "cat_or_dog": cat_or_dog,
                "specie": specie,
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            }

            draw = ImageDraw.Draw(image)
            draw.rectangle((xmin, ymin, xmax, ymax), outline="red")
            image.save(os.path.join(output_folder, image_name))

    return image_details
